# Adjacency Matrix Estimation using DAGMA

This notebook is designed to run in a Python environment having the DAGMA package.

It is recommended to create a dedicated Conda environment for this notebook using the following command:

```bash
conda create --name dagma python=3.x dagma
```

In [ ]:
# !pip install dagma

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import networkx as nx
from dagma import utils
from dagma.linear import DagmaLinear
from utils.data.functions import load_features, load_adjacency_matrix

c:\Users\m.amintoosi\.conda\envs\pth-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
time_start = time.time()


In [4]:
###### Settings ######
DATA_PATHS = {
    "shenzhen": {"feat": "data/sz_speed.csv", "adj": "data/sz_adj.csv"},
    "losloop": {"feat": "data/los_speed.csv", "adj": "data/los_adj.csv"},
}

datasets = ['shenzhen', 'losloop']
use_gsl = False
# use_gsl = True
pred_list = [1, 2, 3, 4]

for dataset_name in datasets:
    print(f"Processing dataset: {dataset_name}")

    ###### Load data ######
    data = load_features(DATA_PATHS[dataset_name]["feat"])
    adj = load_adjacency_matrix(DATA_PATHS[dataset_name]["adj"])

    time_len = data.shape[0]
    num_nodes = data.shape[1]
    data1 = np.mat(data, dtype=np.float32)

    #### Normalization ####
    max_value = np.max(data1)
    data1 = data1 / max_value

    W_true = adj.copy()

    for pre_len in pred_list:
        W_est_file_name = f"data/W_est_{dataset_name}_pre_len{pre_len}.npy"
        if use_gsl:
            #### Estimate adjacency matrix by GSL ####
            W_est_all = np.zeros((num_nodes, num_nodes, pre_len))
            for i in range(pre_len):
                X = data[i::pre_len]
                model = DagmaLinear(loss_type='l2')
                w_est = model.fit(X, lambda1=0.02)
                W_est_all[:, :, i] = w_est

                acc = utils.count_accuracy(W_true, w_est > 0)
                print(f"Prediction length {pre_len}, shape {w_est.shape}, accuracy {acc}, non-zero elements {np.count_nonzero(w_est > 0)}")
            np.save(W_est_file_name, W_est_all)

        else:
            W_est_all = np.load(W_est_file_name)

        #### Combine estimated adjacency matrices ####
        adj_all = np.zeros(W_est_all.shape, dtype=int)
        adj_all[W_est_all > 0] = 1
        adj = np.any(adj_all, axis=2)
        W_est = adj.astype(int)
        print(f"Prediction length {pre_len}, shape {W_est.shape}, non-zero elements {np.count_nonzero(W_est_all > 0)}")
            
        # print(f"adj.shape= {adj.shape}, type= {type(adj[0, 0])}")
        # print(f"W_est.shape= {W_est_all.shape}, type= {type(W_est[0, 0])}")

        # ### Plot the estimated and true adjacency matrix ####   

        # fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # axs[0].imshow(W_true.astype(np.uint8), cmap='gray')
        # axs[0].set_title('W_true')

        # axs[1].imshow(W_est, cmap='gray')
        # axs[1].set_title('adj: W_est')

        # overlay_image = np.zeros((W_true.shape[0], W_true.shape[1], 3), dtype=np.uint8)
        # overlay_image[..., 0] = W_true * 255  # Red channel
        # overlay_image[..., 1] = W_est * 255   # Green channel

        # axs[2].imshow(overlay_image)
        # axs[2].set_title('Overlayed')

Processing dataset: shenzhen
Prediction length 1, shape (156, 156), non-zero elements 0
Prediction length 2, shape (156, 156), non-zero elements 0
Prediction length 3, shape (156, 156), non-zero elements 0
Prediction length 4, shape (156, 156), non-zero elements 0
Processing dataset: losloop
Prediction length 1, shape (207, 207), non-zero elements 28
Prediction length 2, shape (207, 207), non-zero elements 54
Prediction length 3, shape (207, 207), non-zero elements 79
Prediction length 4, shape (207, 207), non-zero elements 110
